In [1]:
# ! pip install ta

## Import

In [1]:
from env.EpisodeData import *

In [1]:
import pickle
import matplotlib.pyplot as plt
import warnings

from config import CONFIG
from easydict import EasyDict

from env.BasicEnv import *
from state.state import *
from agent.PPOAgent_ms import *
from models.CTTS import *
from trainer.nonEpisodic import *
from trainer.Episodic import *
from datahandler.scaler import *
from utils.setDevice import *
from utils.timestepRelated import *
from visualization.methods import *

In [2]:
warnings.filterwarnings("ignore", category=FutureWarning)

## Load

In [3]:
with open('/home/tonnonssi/YOLO-Futures/data/processed/kospi200_ffill_clean_version.pkl', 'rb') as f:
    df = pickle.load(f)

print(df.shape)
df.head(5)

(941402, 8)


,date,time,open,high,low,close,prevClose,vol
2010-02-16 09:01:00,20100216,901,207.55,207.65,207.50,207.60,207.5,3985.0
2010-02-16 09:02:00,20100216,902,207.60,207.65,207.25,207.55,207.5,5095.0
2010-02-16 09:03:00,20100216,903,207.55,207.80,207.50,207.60,207.5,2175.0
2010-02-16 09:04:00,20100216,904,207.55,207.85,207.55,207.80,207.5,1301.0
2010-02-16 09:05:00,20100216,905,207.80,208.15,207.80,208.05,207.5,3870.0


## Set State info • scaler

In [6]:
# target_values = ['open', 'high', 'low', 'close', 
#                 'vol','return_5', 'return_10', 'volume_change', 'ema_5', 
#                 'ema_20', 'ema_cross', 'cci', 'sar', '%K', 
#                 '%D', 'roc', 'rsi', 'obv', 'ad_line', 
#                 'bb_upper', 'bb_lower', 'bb_width', 'atr', 'gap_size']

target_values = ['close', 'high', 'low',
                'ema_5', 'ema_20', 'ema_cross',
                'rsi', '%K', '%D', 'cci',
                'atr', 'bb_width',
                'obv', 'volume_change']

scaler = RobustScaler()

## Set Env info 

In [7]:
train_valid_timestep = split_date_ranges_by_group(df.index, n_group=15, train_ratio=0.9) # [:70000]
train_valid_timestep 

[(('2010-02-16', '2010-09-17'), ('2010-09-20', '2010-10-15')),
 (('2010-10-18', '2011-05-23'), ('2011-05-24', '2011-06-16')),
 (('2011-06-17', '2012-01-18'), ('2012-01-19', '2012-02-14')),
 (('2012-02-15', '2012-09-18'), ('2012-09-19', '2012-10-15')),
 (('2012-10-16', '2013-05-22'), ('2013-05-23', '2013-06-17')),
 (('2013-06-18', '2014-01-23'), ('2014-01-24', '2014-02-19')),
 (('2014-02-20', '2014-09-29'), ('2014-09-30', '2014-10-24')),
 (('2014-10-27', '2015-06-03'), ('2015-06-04', '2015-06-26')),
 (('2015-06-29', '2016-02-01'), ('2016-02-02', '2016-02-29')),
 (('2016-03-02', '2016-10-07'), ('2016-10-10', '2016-11-01')),
 (('2016-11-02', '2017-06-09'), ('2017-06-12', '2017-07-04')),
 (('2017-07-05', '2018-02-12'), ('2018-02-13', '2018-03-12')),
 (('2018-03-13', '2018-10-22'), ('2018-10-23', '2018-11-14')),
 (('2018-11-15', '2019-06-25'), ('2019-06-26', '2019-07-18')),
 (('2019-07-19', '2020-03-10'), ('2020-03-11', '2020-04-03'))]

In [7]:
device = get_device() # torch.device("cpu")  

CONFIG.INPUT_DIM = len(target_values)
CONFIG.REWARD_FTN = risk_adjusted_pnl_reward
CONFIG.DONE_FTN = is_day_changed
CONFIG.TRAIN_VALID_TIMESTEP = train_valid_timestep
CONFIG.SCALER = scaler

In [8]:
state =  State(target_values)

model = MultiStatePV(
    input_dim=CONFIG.INPUT_DIM,
    agent_input_dim=CONFIG.AGENT_INPUT_DIM,
    embed_dim=CONFIG.EMBED_DIM,
    kernel_size=CONFIG.KERNEL_SIZE,
    stride=CONFIG.STRIDE,
    action_size=CONFIG.N_ACTIONS,
    device=device,
    agent_hidden_dim=CONFIG.AGENT_HIDDEN_DIM,
    agent_out_dim=CONFIG.AGENT_OUT_DIM,
    fusion_hidden_dim=CONFIG.FUSION_HIDDEN_DIM,
    num_layers=CONFIG.NUM_LAYERS,
    num_heads=CONFIG.NUM_HEADS,
    d_ff=CONFIG.D_FF,
    dropout=CONFIG.DROPOUT
)

agent = PPOAgent(
    action_space=CONFIG.ACTION_SPACE,
    n_actions=CONFIG.N_ACTIONS,
    model=model,
    value_coeff=CONFIG.VALUE_COEFF,
    entropy_coeff=CONFIG.ENTROPY_COEFF,
    clip_eps=CONFIG.CLIP_EPS,
    gamma=CONFIG.GAMMA,
    lr=CONFIG.LR,
    batch_size=CONFIG.BATCH_SIZE,
    epoch=CONFIG.EPOCH,
    device=device
)

In [ ]:
trainer = EpisodicTrainer( # NonEpisodicTrainer
    df=df,
    env=FuturesEnvironment,
    train_valid_timestep=CONFIG.TRAIN_VALID_TIMESTEP,
    window_size=CONFIG.WINDOW_SIZE,
    state=state,
    reward_ftn=CONFIG.REWARD_FTN,
    done_ftn=CONFIG.DONE_FTN,
    start_budget=CONFIG.START_BUDGET,
    scaler=CONFIG.SCALER,
    position_cap=CONFIG.POSITION_CAP,
    agent=agent,
    model=model,
    optimizer=optim.Adam,
    device=CONFIG.DEVICE,
    n_steps=CONFIG.N_STEPS,
    ma_interval=CONFIG.MA_INTERVAL,
    save_interval=CONFIG.SAVE_INTERVAL,
    path=CONFIG.PATH
)

trainer.save(CONFIG)
trainer()

Robust Scaling Completed.
Robust Scaling Completed.


/home/tonnonssi/Desktop/YOLO-Futures/code/trainer/nonEpisodic.py:344: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  _, ax = plt.subplots(figsize=(12,6))


📁 1. Account Status (계좌 상태)
⏱️  Current Timestep   : 2010-03-04 13:59:00
💰  Available Balance  : 6,995,557 KRW
💼  Margin Deposit     : 11,090,100 KRW
💸  Transaction Costs  : 6,111,843 KRW
📉  Unrealized PnL     : -80,000 KRW
💵  Realized PnL       : -8,827,194 KRW
💰  Total Equity       : 6,915,557 KRW
⚖️  Avg Entry Price    : 211.24
💼  Current Position   : short (-1)
📊  Execution Strength : 10/10
🔢  Total Trades       : 394
📁 2. Performance Metrics (성과 지표)
💵  Total Return       : -76.95%
🏆  Episode Win Rate   : 0.0% (0/175)
🎯  Trade Win Rate     : 44.3% (143/323)
📊  Sharpe Ratio       : 0.002
📉  Max Drawdown       : -77.9%
📁 3. Trade History (거래 기록)
✅  Completed Trades   : 323
💰  Episode PnL        : -4,674,694 KRW
💹  Last Trade PnL     : 31,164 KRW
🔢  Total Actions      : 394
📁 4. Market Conditions (시장 상태)
📈  Previous Close     : 211.40
🌍  Market Regime      : SIDEWAYS
ℹ️  Info Status        : 

📁 1. Account Status (계좌 상태)
⏱️  Current Timestep   : 2010-03-16 10:39:00
💰  Available Balanc

/home/tonnonssi/Desktop/YOLO-Futures/code/trainer/nonEpisodic.py:350: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig(path)


📁 1. Account Status (계좌 상태)
⏱️  Current Timestep   : 2010-03-08 10:39:00
💰  Available Balance  : 18,690,802 KRW
💼  Margin Deposit     : 1,124,025 KRW
💸  Transaction Costs  : 4,535,174 KRW
📉  Unrealized PnL     : 0 KRW
💵  Realized PnL       : -7,914,932 KRW
💰  Total Equity       : 18,690,802 KRW
⚖️  Avg Entry Price    : 214.10
💼  Current Position   : short (-1)
📊  Execution Strength : 1/10
🔢  Total Trades       : 392
📁 2. Performance Metrics (성과 지표)
💵  Total Return       : -37.70%
🏆  Episode Win Rate   : 2.7% (14/517)
🎯  Trade Win Rate     : 43.1% (137/318)
📊  Sharpe Ratio       : 0.321
📉  Max Drawdown       : -66.8%
📁 3. Trade History (거래 기록)
✅  Completed Trades   : 318
💰  Episode PnL        : -4,122,432 KRW
💹  Last Trade PnL     : -1,363,590 KRW
🔢  Total Actions      : 392
📁 4. Market Conditions (시장 상태)
📈  Previous Close     : 217.00
🌍  Market Regime      : SIDEWAYS
ℹ️  Info Status        : done

📁 1. Account Status (계좌 상태)
⏱️  Current Timestep   : 2010-03-10 13:59:00
💰  Available Bal

In [ ]:
import pandas as pd
import bisect

def find_nearest_later_timestamp(list_of_timesteps, target_time):
    """
    list_of_timesteps: 정렬된 pd.Timestamp 리스트
    target_time: pd.Timestamp
    """
    # 보장: list_of_timesteps는 오름차순 정렬되어 있어야 함
    idx = bisect.bisect_left(list_of_timesteps, target_time)

    if idx < len(list_of_timesteps):
        return list_of_timesteps[idx]
    else:
        # target_time보다 늦은 값이 없음
        return None